<a href="https://colab.research.google.com/github/StetskoSergey/pida5_stetsko/blob/master/f_dz%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas_profiling
from sklearn import preprocessing

from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('aac_shelter_outcomes.csv')
data.head(5)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


Убираем незначащие столбцы - Дату рождения, так как нам важно только возвраст животного до момента передачи, дату убираем, так как нам не важно в какой день произошло

In [0]:
data = data.drop(['animal_id', 'name', 'date_of_birth' , 'datetime' , 'monthyear'], axis = 1)

Переведем возраст животного на момент передачи в количество дней, и удалим пустые строки

In [4]:
data['age_upon_outcome'].unique()

array(['2 weeks', '1 year', '9 years', '5 months', '4 months', '3 years',
       '1 month', '3 months', '2 years', '2 months', '4 years', '8 years',
       '3 weeks', '8 months', '12 years', '7 years', '5 years', '6 years',
       '5 days', '10 months', '4 weeks', '10 years', '2 days', '6 months',
       '14 years', '11 months', '15 years', '7 months', '13 years',
       '11 years', '16 years', '9 months', '3 days', '6 days', '4 days',
       '5 weeks', '1 week', '1 day', '1 weeks', '0 years', '17 years',
       '20 years', '18 years', '19 years', '22 years', '25 years', nan],
      dtype=object)

Уберем отсутствующие значения в колоне Возраст животного

In [0]:
data_age = data.dropna(subset=['age_upon_outcome'])

Конвертируем возраст животного в количество дней

In [0]:
def convert_age(row):
  col = 'age_upon_outcome'
  if row[col].find('day')>0:
    return int(row[col][:row[col].find(' ')])
  elif row[col].find('week')>0:
    return int(row[col][:row[col].find(' ')])*7
  elif row[col].find('month')>0:
    return int(row[col][:row[col].find(' ')])*30
  elif row[col].find('year')>0:
    return int(row[col][:row[col].find(' ')])*365       


In [7]:
data_age['num_age'] = data_age.apply(convert_age, axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
data_age = data_age.drop('age_upon_outcome', axis = 1)

Учитывая что в столбе outcome_type много видов значений, а предсказать нам нужно только 2, то приведем значения в Этом Столбце к трем значениям - 2 требуемым и "Прочие"

In [0]:
def convert_outcome_type(row):
  col = 'outcome_type'
  if row[col]=='Adoption':
    return row[col]
  elif row[col]=='Transfer':
    return row[col]
  else :
    return 'Other'     


In [0]:
data_age['outcome_type'] = data_age.apply(convert_outcome_type, axis = 1)

Заменяем во всех остальных столбцах не заполненные значения на Other

In [0]:
data_age = data_age.fillna('Other')

Уберем дубликаты

In [0]:
data_age = data_age.drop_duplicates()
data_age = data_age.reindex()

In [15]:
data_age.head()

,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome,num_age
0,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male,14
1,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female,365
2,Dog,Pit Bull,Blue/White,Other,Adoption,Neutered Male,365
3,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male,3285
4,Other,Bat Mix,Brown,Rabies Risk,Other,Unknown,150


Остались категориальные колонки - sex_upon_outcome, outcome_subtype, animal_type, breed, color
Сделаем анализ Таблицы

In [16]:
data_age.describe()

,num_age
count,42366.000000
mean,1019.200562
std,1232.831358
min,0.000000
25%,180.000000
50%,365.000000
75%,1460.000000
max,9125.000000


In [34]:
pandas_profiling.ProfileReport(data_age)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,8
Number of observations,42366
Total Missing (%),0.0%
Total size in memory,2.6 MiB
Average record size in memory,64.0 B
Numeric,2
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0


Сделаем OneHotEncoder по всем категориальным столбцам и для этого напишем функцию, делающую преобразования всех категориальных столбцов

In [0]:
def oh_encoder(df, list_col):
  
  r_df = df.copy()

  for col in list_col:
    conv = df[col].get_values().reshape(-1, 1)

    # One-hot encoding
    oh_encoder = preprocessing.OneHotEncoder()
    oh_encoder.fit(conv)
    oh_result = oh_encoder.transform(conv).toarray()

    conv_columns = ['{}_{}'.format(col,i) for i in range(oh_result.shape[1])]
    conv_df = pd.DataFrame(oh_result, columns=conv_columns)
    conv_df.index = df.index
    r_df = pd.concat([r_df, conv_df], axis=1)
    r_df = r_df.drop([col], axis=1)

  return r_df  

In [0]:
list_col = ['outcome_type','sex_upon_outcome','animal_type','outcome_subtype','breed', 'color']

In [27]:
t_data = oh_encoder(data_age, list_col)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  


In [28]:
t_data.head()

,num_age,outcome_type_0,outcome_type_1,outcome_type_2,sex_upon_outcome_0,sex_upon_outcome_1,sex_upon_outcome_2,sex_upon_outcome_3,sex_upon_outcome_4,sex_upon_outcome_5,animal_type_0,animal_type_1,animal_type_2,animal_type_3,animal_type_4,outcome_subtype_0,outcome_subtype_1,outcome_subtype_2,outcome_subtype_3,outcome_subtype_4,outcome_subtype_5,outcome_subtype_6,outcome_subtype_7,outcome_subtype_8,outcome_subtype_9,outcome_subtype_10,outcome_subtype_11,outcome_subtype_12,outcome_subtype_13,outcome_subtype_14,outcome_subtype_15,outcome_subtype_16,outcome_subtype_17,outcome_subtype_18,outcome_subtype_19,breed_0,breed_1,breed_2,breed_3,breed_4,...,color_485,color_486,color_487,color_488,color_489,color_490,color_491,color_492,color_493,color_494,color_495,color_496,color_497,color_498,color_499,color_500,color_501,color_502,color_503,color_504,color_505,color_506,color_507,color_508,color_509,color_510,color_511,color_512,color_513,color_514,color_515,color_516,color_517,color_518,color_519,color_520,color_521,color_522,color_523,color_524
0,14,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,365,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,365,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3285,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,150,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
list_res = ['outcome_type_0','outcome_type_1','outcome_type_2']
train_X = t_data.drop(list_res, axis = 1)
train_Y = t_data[list_res]

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_X,
                                                 train_Y,
                                                 test_size = 0.2,#%80 train, 20% test
                                                 random_state = 3)

In [36]:
from sklearn.ensemble import RandomForestClassifier
# создаем модель деревья решений
model=RandomForestClassifier(n_estimators=100)

#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score 
#model = LogisticRegression()
model.fit(X_train, y_train)
predicted_classes = model.predict(X_test)
accuracy = accuracy_score(y_test,predicted_classes)
print('Точность предсказания модели: ', accuracy)

Точность предсказания модели:  0.7860514514987019


In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

estimator = LinearRegression()
estimator.fit(X_train, y_train) # Обучение
y_pred = estimator.predict(X_test)

print("R2: \t", r2_score(y_test, y_pred))
print("RMSE: \t", sqrt(mean_squared_error(y_test, y_pred)))
print("MAE: \t", mean_absolute_error(y_test, y_pred))


R2: 	 0.638379163264393
RMSE: 	 0.2872427238834524
MAE: 	 0.18869935809435123


Не забывайте, что у вас есть PCA (Метод главных компонент). Он может пригодиться.

Уменьшим количество параметров до 30 ( с 2500),
что сохранит точность, но ускорит вычисления

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 30)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_

In [45]:
explained_variance

array([9.99997575e-01, 3.43105133e-07, 2.27828539e-07, 2.14961599e-07,
       1.04452805e-07, 8.24153973e-08, 6.98179577e-08, 5.78908006e-08,
       5.18837576e-08, 4.16762987e-08, 4.09524743e-08, 3.71609418e-08,
       3.36632032e-08, 2.88886471e-08, 2.84716176e-08, 2.77858616e-08,
       2.38324581e-08, 2.30606189e-08, 2.28204361e-08, 2.22861321e-08,
       2.10966046e-08, 2.08010689e-08, 1.90662452e-08, 1.85101374e-08,
       1.77633512e-08, 1.70357661e-08, 1.60391035e-08, 1.57046270e-08,
       1.46842638e-08, 1.34054906e-08])

In [44]:
from sklearn.ensemble import RandomForestClassifier
# создаем модель деревья решений
model=RandomForestClassifier(n_estimators=100)

#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score 
#model = LogisticRegression()
model.fit(X_train, y_train)
predicted_classes = model.predict(X_test)
accuracy = accuracy_score(y_test,predicted_classes)
print('Точность предсказания модели: ', accuracy)

Точность предсказания модели:  0.7834552749586972
